In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2021, 2026))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i) # so we don't get banned for trying to scrape too fast

        try: 
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector) #only select pieces of html
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue #goes back to beginning of loop @ for statement
        else:
            break #successful scrape = break the loop, not retry
    return html
    
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f: 
            f.write(html)



In [5]:
for season in SEASONS:
    await scrape_season(season)
    

2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2025_games.html
2024-25 NBA Schedule | Basketball-Reference.com
2024-25 NBA Schedule | Basketball-Reference.com
2024-25 NBA Schedule | Basketball-Reference.com
2024-25 NBA Schedule | Basketball-Reference.com


In [6]:
#parsing boxscore links 
standings_files = os.listdir(STANDINGS_DIR)

In [7]:
standings_files

['.ipynb_checkpoints',
 'NBA_2018_games-april.html',
 'NBA_2018_games-december.html',
 'NBA_2018_games-february.html',
 'NBA_2018_games-january.html',
 'NBA_2018_games-june.html',
 'NBA_2018_games-march.html',
 'NBA_2018_games-may.html',
 'NBA_2018_games-november.html',
 'NBA_2018_games-october.html',
 'NBA_2019_games-april.html',
 'NBA_2019_games-december.html',
 'NBA_2019_games-february.html',
 'NBA_2019_games-january.html',
 'NBA_2019_games-june.html',
 'NBA_2019_games-march.html',
 'NBA_2019_games-may.html',
 'NBA_2019_games-november.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-august.html',
 'NBA_2020_games-december.html',
 'NBA_2020_games-february.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-july.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-november.html',
 'NBA_2020_games-october-2019.html',
 'NBA_2020_games-october-2020.html',
 'NBA_2020_games-september.html',
 'NBA_2021_games-april.html',
 'NBA_2021_games-december.html',
 'NBA_2021_games-february

In [8]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(html)

In [9]:
standings_files = [s for s in standings_files if ".html" in s]

In [10]:

for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)


Trail Blazers vs Hawks, April 1, 2025 | Basketball-Reference.com
Suns vs Bucks, April 1, 2025 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202504010NYK.html
76ers vs Knicks, April 1, 2025 | Basketball-Reference.com
Raptors vs Bulls, April 1, 2025 | Basketball-Reference.com
Warriors vs Grizzlies, April 1, 2025 | Basketball-Reference.com
Magic vs Spurs, April 1, 2025 | Basketball-Reference.com
Timberwolves vs Nuggets, April 1, 2025 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202504020CLE.html
Knicks vs Cavaliers, April 2, 2025 | Basketball-Reference.com
Hornets vs Pacers, April 2, 2025 | Basketball-Reference.com
Kings vs Wizards, April 2, 2025 | Basketball-Reference.com
Heat vs Celtics, April 2, 2025 | Basketball-Reference.com
Jazz vs Rockets, April 2, 2025 | Basketball-Reference.com
Hawks vs Mavericks, April 2, 2025 | Basketball-Reference.com
Spurs vs Nuggets, April 2, 2025 | Basketball-Referenc